<h2><center>IIC2440 – Procesamiento de Datos Masivos</center></h2>
<h3><center>Tarea 1: Análisis sesiones parlamentarias</center></h3>


**Integrantes:**
- Franco Chiappe
- Tomás Pérez

### Librerias

In [2]:
# Importar librerias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import unicodedata

## Parte 1 [1 pto] - Procesando los datos

In [3]:
# Cargar los datos
parlamentarios_info = pd.read_csv('participacion/parlamentarios_info_general.csv')
descripcion_2023_1 = pd.read_csv('participacion/participacion_descripcion_2023_1.csv')
descripcion_2023_2 = pd.read_csv('participacion/participacion_descripcion_2023_2.csv')
descripcion_2024 = pd.read_csv('participacion/participacion_descripcion_2024.csv')
df = pd.concat([descripcion_2023_1, descripcion_2023_2, descripcion_2024], ignore_index=True)

### Pre-Procesamiento de Datos

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68161 entries, 0 to 68160
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    68161 non-null  int64  
 1   PARLAMENTARIO_ID    68161 non-null  int64  
 2   FECHA               68161 non-null  object 
 3   LEGISLATURA         68161 non-null  int64  
 4   SESION              68161 non-null  int64  
 5   TIPO_SESION         68161 non-null  object 
 6   TIPO_PARTICIPACION  68161 non-null  object 
 7   CAMARA              68161 non-null  object 
 8   PARLAMENTARIOS      68161 non-null  object 
 9   TEXTO_PRINCIPAL     68161 non-null  object 
 10  TEXTO_ANTECEDENTE   32393 non-null  object 
 11  DESCRIPCION_DEBATE  68161 non-null  object 
 12  BOLETIN_ID          20593 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 6.8+ MB


Nos damos cuenta que en las columnas:

- `TEXTO_ANTECEDENTE` tiene 35768  datos nulos
- `BOLETIN_ID` tiene 47568 datos nulos

Decidimos eliminar la columna `TEXTO_ANTECEDENTE`, porque presentaba un alto porcentaje de datos nulos y no aportaba informacion relevante. Por otro parte, no eliminamos la columna `BOLETIN_ID`, esta columna podria resultar util para analisis futuros.

In [5]:
df = df.drop(columns=["TEXTO_ANTECEDENTE"])

### Limpieza de texto

In [6]:
def limpiar_texto(texto):
        
    texto = texto.lower()
    texto = texto.strip()
    
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

    return texto

Se usa la funcion ***limpiar_texto*** el objetivo es normalizar los textos para facilitar la extraccion de keywords. Esta funcion convierte todo el contenido a minuscula, elimina tildes, signos de puntuación y espacios innecesarios, dejando un text limpio. Esto permite detectar con mayor precision los temas.

In [7]:
# Aplicar la funcion de limpieza a las columnas
df['DESCRIPCION_DEBATE'] = df['DESCRIPCION_DEBATE'].apply(limpiar_texto)
df['TEXTO_PRINCIPAL'] = df['TEXTO_PRINCIPAL'].apply(limpiar_texto)

# Juntamos las columnas
df["TEXTO_COMBINADO"] = df["TEXTO_PRINCIPAL"] + " " + df["DESCRIPCION_DEBATE"]
df["TEXTO_COMBINADO"].head()

0    la senora allende ngracias presidente npor su ...
1    la senora allende ngracias presidente nen prim...
2    proyecto de acuerdo de los honorables senadore...
3    de la senora allende n a la municipalidad de v...
4    proyecto de acuerdo de los honorables senadore...
Name: TEXTO_COMBINADO, dtype: object

### Temas y palabras claves

In [8]:
temas = {
    "salud": ["salud", "hospital", "hospitales" "medico", "vacuna", "enfermedad", "medicina", "medicamento"],
    "educacion": ["educacion", "colegio", "escuela", "universidad", "profesor", "alumno", "estudiante", "educador", "colegios"],
    "delincuencia": ["delincuencia", "carabinero", "robo", "narcotrafico", "seguridad", "delito", "delincuente", "delincuentes"],
    "medio ambiente": ["medioambiente", "naturaleza", "contaminacion", "reciclaje", "agua", "bosque", "bosques", "flora", "fauna", "ecosistema", "ecosistemas"],
    "economia": ["economia", "dinero", "impuesto", "trabajo", "impuestos", "empresa", "empresas", "negocio", "negocios", "inversion", "inversiones", "financiero", "financieros"],
    "constitucion": ["constitucion", "constitucional"],
    "vivienda": ["vivienda", "casa", "alquiler", "arrendar"],
    "transporte": ["transporte", "metro", "camino", "micro"],
    "derechos humanos": ["derechos humanos", "derechos", "humanos", "discriminacion", "discriminacion", "discriminados", "discriminados", "discriminacion", "discriminaciones", "humanitaria"],
    "legislacion": ["legislacion", "ley", "leyes", "legislativo", "legislativa"],
    "justicia": ["justicia", "justicia", "jueces", "juez", "tribunal", "tribunales"],
}

In [9]:
def asignar_keywords(texto):
    palabras = texto.split()
    etiquetas = set()
    for tema, claves in temas.items():
        for palabra in claves:
            if (palabra in palabras):
                etiquetas.add(tema)
       
    return list(etiquetas)

In [10]:
# Se agrupan por descripción debate, para poder aplicar la función de asignar_keywords a cada grupo de manera más eficiente
grouped_df = df.groupby("DESCRIPCION_DEBATE").first().reset_index()
grouped_df["KEYWORDS"] = grouped_df["TEXTO_COMBINADO"].apply(asignar_keywords)

df = df.merge(grouped_df[["TEXTO_COMBINADO", "KEYWORDS"]], on="TEXTO_COMBINADO", how="left")

### Conclusion:



Se realizo el preprocesamiento de las intervenciones parlamentarias con el objetivo de identificar las tematicas. Para esto, se aplico una funcion de limpieza que normalizo el texto (conversion a minusculas, eliminacion de tildes, puntuacion y espacios innecesarios). Luego, se unieron las columnas `TEXTO_ANTECEDENTE` y `DESCRIPCION_DEBATE` para generar un texto combinado. Finalmente, se aplico una funcion que detecta las palabras clave y asigna a cada intervencion una o mas temas, permitiendo categorizar los discursos parlamentarios segun temas como salud, educación, delincuencia, entre otros. Para optimizar la asignación, se agruparon las filas por la columna `DESCRIPCION_DEBATE`, ya que muchas compartian o eran de un mismo debate, para así acortar el tiempo de ejecución de asignación

## Parte 2 - [1 pts] - Modelamiento

<h1> <center> Para esta parte vamos a trabajar con SQL <h1>


### Revisión Tipo de Datos

In [11]:
parlamentarios_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PARLAMENTARIO_ID       1199 non-null   int64  
 1   NOMBRE_COMPLETO        1199 non-null   object 
 2   VIGENTE                1199 non-null   int64  
 3   FECHA_NACIMIENTO       206 non-null    object 
 4   REDES_SOCIALES         0 non-null      float64
 5   SITIOS_WEB             0 non-null      float64
 6   CAMARA_ACTUAL          206 non-null    object 
 7   CAMARA                 206 non-null    object 
 8   INICIO                 206 non-null    float64
 9   FINAL                  206 non-null    float64
 10  PARTIDO_POLITICO       155 non-null    object 
 11  UNIDAD_QUE_REPRESENTA  205 non-null    object 
dtypes: float64(4), int64(2), object(6)
memory usage: 112.5+ KB


In [12]:
# df con las keys
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68161 entries, 0 to 68160
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    68161 non-null  int64  
 1   PARLAMENTARIO_ID    68161 non-null  int64  
 2   FECHA               68161 non-null  object 
 3   LEGISLATURA         68161 non-null  int64  
 4   SESION              68161 non-null  int64  
 5   TIPO_SESION         68161 non-null  object 
 6   TIPO_PARTICIPACION  68161 non-null  object 
 7   CAMARA              68161 non-null  object 
 8   PARLAMENTARIOS      68161 non-null  object 
 9   TEXTO_PRINCIPAL     68161 non-null  object 
 10  DESCRIPCION_DEBATE  68161 non-null  object 
 11  BOLETIN_ID          20593 non-null  float64
 12  TEXTO_COMBINADO     68161 non-null  object 
 13  KEYWORDS            19803 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 7.3+ MB


### Las tablas son: 

Son 7 tablas las cuales son:

- `Parlamentarios`
- `Partidos_Politico`
- `Participaciones`
- `Debates`
- `Intervenciones`
- `Keywords`
- `Intervenciones_Keywords`

```sql
Table Parlamentarios {
  PARLAMENTARIO_ID integer [pk]
  NOMBRE_COMPLETO varchar
  VIGENTE boolean
  FECHA_NACIMIENTO date
  CAMARA_ACTUAL varchar
  CAMARA varchar
  INICIO integer
  FINAL integer
  UNIDAD_QUE_REPRESENTA varchar
  PARTIDO_ID integer [ref: > Partidos_Politico.PARTIDO_ID]
}
```

`Parlamentarios`
Contiene informacion basica sobre los parlamentarios, como su nombre, fecha de nacimiento, camara a la que pertenecen, si estan vigentes, y el partido politico al que representan. Es fundamental para asociar intervenciones y participaciones con un parlamentario especifico y su afiliacion politica.

```sql
Table Partidos_Politico {
  NOMBRE_PARTIDO varchar 
  PARTIDO_ID integer [pk]
}
```

`Partidos_Politico`
Registra los partidos politicos con su nombre y un identificador. Esta tabla permite agrupar y analizar las intervenciones por politico, lo cual es importante para consultas como la media mpvil por partido y temas mas tratados por cada partido.



```sql
Table Debates {
  DEBATE_ID integer [pk]
  DESCRIPCION_DEBATE varchar
  FECHA timestamp
  BOLETIN_ID varchar
}
```

`Debates`
Incluye los debates realizados, con su fecha, descripcion y un boletin identificador. Es el marco temporal y tematico donde ocurren las participaciones parlamentarias. Es clave para organizar las intervenciones por mes o sesion.


```sql
Table Participaciones {
  ID_PARTICIPACION integer [pk]
  PARLAMENTARIO_ID integer [ref: > Parlamentarios.PARLAMENTARIO_ID]
  DEBATE_ID integer [ref: > Debates.DEBATE_ID]
  FECHA timestamp
  LEGISLATURA integer
  SESION integer
  TIPO_SESION varchar
  TIPO_PARTICIPACION varchar
  CAMARA varchar
}
```
`Participaciones`
Vincula a los parlamentarios con los debates en los que han participado. Tambiin indica la legislatura, sesion, tipo de participacion y camara. Esta tabla permite identificar cuando y como participo un parlamentario, lo cual es base para calcular promedios y frecuencias en el tiempo.


```sql
Table Keywords {
  KEYWORD_ID integer [pk]
  nombre varchar
}
```
`Keywords`
Define las tematicas o palabras clave que se usan para clasificar las intervenciones. Son importantes para responder consultas como las tematicas mas tratadas en un mes o meses.


```sql
Table Intervenciones {
  INTERVENCION_ID integer [pk]
  ID_PARTICIPACION integer [ref: > Participaciones.ID_PARTICIPACION]
  TEXTO_PRINCIPAL varchar
}
```
`Intervenciones`
Guarda los textos especificos pronunciados por los parlamentarios en cada participacion. Es el contenido discursivo que luego se puede analizar tematicamente.

```sql
Table Intervenciones_Keywords {
  INTERVENCION_ID integer [ref: > Intervenciones.INTERVENCION_ID]
  KEYWORD_ID integer [ref: > Keywords.KEYWORD_ID]
}
```

`Intervenciones_Keywords`
Tabla de relación entre intervenciones y palabras clave. Permite saber que temáticas estan presentes en que intervenciones, lo cual es central para analisis temzticos y rankings de temas tratados.

### ***Conuslta 1:*** Cuales son las top 5 tematicas mas tratadas, para cada mes en el que hay registros.

<h4> El codigo es: </h4>

```sql
WITH conteo_keywords AS (
  SELECT
  FORMAT_DATE('%Y-%m', FECHA) AS mes,
    Keywords.KEYWORD,
    COUNT(*) AS cantidad
  FROM
    `Sesiones_Parlamentarias`.`Debates` AS Debates
  LEFT JOIN
    `Sesiones_Parlamentarias`.`Intervenciones_Keywords` AS t0
  ON
    Debates.DEBATE_ID = t0.INTERVENCION_ID
  LEFT JOIN
    `Sesiones_Parlamentarias`.`Keywords` AS Keywords
  ON
    t0.KEYWORD_ID = Keywords.KEYWORD_ID
  GROUP BY
    mes,
    Keywords.KEYWORD
),
ranked_keywords AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY mes ORDER BY cantidad DESC) AS rank
  FROM
    conteo_keywords
)
SELECT
  mes,
  KEYWORD,
  cantidad
FROM
  ranked_keywords
WHERE
  rank <= 5
ORDER BY
  mes,
  rank;
```



Esta consulta SQL tiene como objetivo identificar las cinco temáticas más tratadas (palabras clave o keywords) en las intervenciones parlamentarias, separadas por mes. Para ello, se agrupan las intervenciones por mes y palabra clave, y se cuenta cuántas veces aparece cada una. Luego, se ordenan por frecuencia dentro de cada mes y se asigna un ranking. Finalmente, se filtran las cinco primeras por mes, entregando así un resumen mensual de las palabras más recurrentes en el debate parlamentario, lo que permite identificar los temas más relevantes discutidos en cada periodo.




### ***Consulta 2:*** Media movil de intervenciones por partido politico con un intervalo de 3 meses.


<h4> El codigo es: </h4>


```sql
SELECT
    PARTIDO_POLITICO,
    ROUND(AVG(promedio_individual), 2) AS promedio_intervenciones
FROM (
    SELECT
        t0.PARTIDO_POLITICO,
        Parlamentarios.PARLAMENTARIO_ID,
        AVG(COUNT(t2.INTERVENCION_ID)) OVER (PARTITION 
        BY Parlamentarios.PARLAMENTARIO_ID) AS promedio_individual
    FROM
        `Sesiones_Parlamentarias`.`Partidos_Politico` AS t0
    INNER JOIN
        `Sesiones_Parlamentarias`.`Parlamentarios` AS Parlamentarios
        ON t0.PARTIDO_ID = Parlamentarios.PARTIDO_ID
    INNER JOIN
        `Sesiones_Parlamentarias`.`Participaciones` AS t1
        ON Parlamentarios.PARLAMENTARIO_ID = t1.PARLAMENTARIO_ID
    INNER JOIN
        `Sesiones_Parlamentarias`.`Intervenciones_Keywords` AS t2
        ON t1.ID_PARTICIPACION = t2.INTERVENCION_ID
    GROUP BY
        t0.PARTIDO_POLITICO, Parlamentarios.PARLAMENTARIO_ID
) AS sub
GROUP BY
    PARTIDO_POLITICO
HAVING
    COUNT(*) >= 3;
```

Esta consulta calcula la media móvil de intervenciones por partido político utilizando un intervalo de 3 meses. Primero, para cada parlamentario, se calcula su promedio individual de intervenciones a lo largo del tiempo. Luego, estos promedios individuales se agrupan por partido político, promediándolos para obtener un valor representativo del comportamiento del partido completo. Finalmente, se filtran los partidos que tienen datos de al menos tres parlamentarios, lo que asegura que el promedio sea significativo. Esta métrica permite comparar la actividad parlamentaria entre partidos considerando la variación individual de sus miembros y suavizando los datos a través del tiempo.


### ***Consulta 3:*** Para cada trimestre, ver el tema principal tratado por cada partido politico.

<h4> El codigo es: </h4>


```sql

SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(Debates.FECHA, QUARTER)) AS trimestre,
  t1.PARTIDO_POLITICO,
  Keywords.KEYWORD,
  COUNT(*) AS frecuencia
FROM
  `Sesiones_Parlamentarias`.`Debates` AS Debates
INNER JOIN
  `Sesiones_Parlamentarias`.`Participaciones` AS t0
  ON Debates.DEBATE_ID = t0.DEBATE_ID
INNER JOIN
  `Sesiones_Parlamentarias`.`Parlamentarios` AS Parlamentarios
  ON t0.PARLAMENTARIO_ID = Parlamentarios.PARLAMENTARIO_ID
INNER JOIN
  `Sesiones_Parlamentarias`.`Partidos_Politico` AS t1
  ON Parlamentarios.PARTIDO_ID = t1.PARTIDO_ID
INNER JOIN
  `Sesiones_Parlamentarias`.`Intervenciones_Keywords` AS t2
  ON t0.ID_PARTICIPACION = t2.INTERVENCION_ID
INNER JOIN
  `Sesiones_Parlamentarias`.`Keywords` AS Keywords
  ON t2.KEYWORD_ID = Keywords.KEYWORD_ID
GROUP BY
  trimestre, t1.PARTIDO_POLITICO, Keywords.KEYWORD
),

RankingTemas AS (
  SELECT *,
    ROW_NUMBER() OVER (
      PARTITION BY trimestre, PARTIDO_POLITICO
      ORDER BY frecuencia DESC
    ) AS orden
  FROM ConteoTemas
)

SELECT
  trimestre,
  PARTIDO_POLITICO,
  KEYWORD AS tema_principal,
  frecuencia
FROM
  RankingTemas
WHERE
  orden = 1
ORDER BY
  trimestre, PARTIDO_POLITICO;
```


La consulta busca identificar el tema principal tratado por cada partido político en cada trimestre. Primero, agrupa los debates por trimestre, partido político y tema, contando las menciones de cada tema. Luego, usa `ROWNUMBER()` para clasificar los temas por frecuencia dentro de cada partido y trimestre. Finalmente, selecciona el tema más frecuente de cada partido en cada trimestre, mostrando el resultado ordenado por trimestre y partido. Esto permite ver qué temas son más relevantes para cada partido en cada periodo.


### ***Consulta 4:*** Poder pararse en un mes y ver cual es el top 3 de tematicas tratada por cada partido. Esto vas a querer repetirlo para algunos meses

<h4> El codigo es: </h4>



```sql

WITH ConteoMensual AS (
  SELECT
    FORMAT_DATE('%Y-%m', Debates.FECHA) AS mes,
    t1.PARTIDO_POLITICO,
    Keywords.KEYWORD,
    COUNT(*) AS cantidad
  FROM
    `Sesiones_Parlamentarias`.`Debates` AS Debates
  INNER JOIN
    `Sesiones_Parlamentarias`.`Participaciones` AS t0
    ON Debates.DEBATE_ID = t0.DEBATE_ID
  INNER JOIN
    `Sesiones_Parlamentarias`.`Parlamentarios` AS Parlamentarios
    ON t0.PARLAMENTARIO_ID = Parlamentarios.PARLAMENTARIO_ID
  INNER JOIN
    `Sesiones_Parlamentarias`.`Partidos_Politico` AS t1
    ON Parlamentarios.PARTIDO_ID = t1.PARTIDO_ID
  INNER JOIN
    `Sesiones_Parlamentarias`.`Intervenciones_Keywords` AS t2
    ON t0.ID_PARTICIPACION = t2.INTERVENCION_ID
  INNER JOIN
    `Sesiones_Parlamentarias`.`Keywords` AS Keywords
    ON t2.KEYWORD_ID = Keywords.KEYWORD_ID
  WHERE
    FORMAT_DATE('%Y-%m', Debates.FECHA) = '2023-06'  -- Cambiar mes
  GROUP BY
    mes, t1.PARTIDO_POLITICO, Keywords.KEYWORD
),

Top3Temas AS (
  SELECT *,
    ROW_NUMBER() OVER (
      PARTITION BY PARTIDO_POLITICO
      ORDER BY cantidad DESC
    ) AS orden
  FROM ConteoMensual
)

SELECT
  mes,
  PARTIDO_POLITICO,
  KEYWORD,
  cantidad
FROM
  Top3Temas
WHERE
  orden <= 3
ORDER BY
  PARTIDO_POLITICO,
  orden;
```


La consulta tiene como objetivo obtener el top 3 de temáticas más tratadas por cada partido político en un mes específico. Primero, agrupa los debates por mes, partido político y tema, contando las menciones de cada tema en ese mes. Luego, utiliza la función `ROWNUMBER()` para ordenar los temas dentro de cada partido según su frecuencia, asignando un número de orden. Finalmente, selecciona los tres temas principales de cada partido (los primeros tres con más menciones) y los muestra ordenados por partido político y número de orden. La consulta permite repetir este proceso para diferentes meses cambiando el valor en la cláusula `WHERE` de la fecha (`2023-06` en este caso).


## Parte 3 - [1.5 pts] - BigQuery

Crear los dataframes necesarios a partir de los CSV existentes, borrando los datos nulos

In [13]:
partidos = parlamentarios_info[['PARTIDO_POLITICO']].drop_duplicates().dropna().reset_index(drop=True)
partidos.columns = ['NOMBRE_PARTIDO']
df_debates = df[["DESCRIPCION_DEBATE", "TEXTO_COMBINADO"]].drop_duplicates()

df_intervenciones = df.drop(columns=["KEYWORDS"])
df_intervenciones_keywords = df[["TEXTO_COMBINADO", "KEYWORDS"]].explode("KEYWORDS").drop_duplicates()

In [14]:
partidos.head()

,NOMBRE_PARTIDO
0,Partido Unión Demócrata Independiente
1,Partido Comunista de Chile
2,Partido Republicano de Chile
3,Unión Demócrata Independiente
4,Partido Por la Democracia


<h3> Creamos la dataframes Parlamentarios </h3>

In [15]:
parlamentarios_info["PARTIDO_POLITICO"] = parlamentarios_info["PARTIDO_POLITICO"].fillna("Independiente")
parlamentarios = parlamentarios_info.copy()
parlamentarios = parlamentarios.drop(columns=["REDES_SOCIALES", "SITIOS_WEB", "PARTIDO_POLITICO"])

<h3> Creamos la tabla Partidos_Politico  </h3>

In [16]:
partidos = parlamentarios_info[["PARTIDO_POLITICO"]].drop_duplicates().reset_index(drop=True)
partidos["PARTIDO_ID"] = range(1, len(partidos) + 1)
partidos.columns = ["PARTIDO_POLITICO", "PARTIDO_ID"]
partido_to_id = partidos.set_index("PARTIDO_POLITICO")["PARTIDO_ID"].to_dict()

<h3> Creamos la dataframes  Debates </h3>


In [17]:
debates = df.copy()
debates["DEBATE_ID"] = debates.groupby(["DESCRIPCION_DEBATE"]).ngroup() + 1
debates["BOLETIN_ID"] = debates["BOLETIN_ID"].fillna(0).astype(int)

debates = debates[["DEBATE_ID", "DESCRIPCION_DEBATE", "FECHA", "BOLETIN_ID"]]
debates = debates.drop_duplicates(subset=["DESCRIPCION_DEBATE"])

debates = debates.sort_values(by="DEBATE_ID").reset_index(drop=True)

<h3> Creamos la dataframes Participaciones </h3>


In [18]:
participaciones = df.copy()

debate_to_id = debates.set_index("DESCRIPCION_DEBATE")["DEBATE_ID"].to_dict()
participaciones["DEBATE_ID"] = participaciones["DESCRIPCION_DEBATE"].map(debate_to_id)

participaciones = participaciones[[ "ID_PARTICIPACION", "PARLAMENTARIO_ID", "DEBATE_ID", "FECHA", "LEGISLATURA", "SESION", "TIPO_SESION", "TIPO_PARTICIPACION", "CAMARA"]]

participaciones = participaciones.sort_values(by="ID_PARTICIPACION").reset_index(drop=True)

<h3> Creamos la dataframes Keywords </h3>


In [19]:
keywords = pd.DataFrame(list(temas.items()), columns=["Tema", "Palabras_Clave"])
keywords.rename(columns={"Tema": "KEYWORD"}, inplace=True)
keywords.drop(columns=["Palabras_Clave"], inplace=True)
keywords["KEYWORD_ID"] = range(1, len(keywords) + 1)

<h3> Creamos la dataframes Intervenciones </h3>


In [20]:
intervenciones = df[["ID_PARTICIPACION", "TEXTO_PRINCIPAL"]].copy()
intervenciones["INTERVENCION_ID"] = range(1, len(intervenciones) + 1)
intervenciones = intervenciones[["INTERVENCION_ID", "ID_PARTICIPACION", "TEXTO_PRINCIPAL"]]
intervenciones = intervenciones.sort_values(by="INTERVENCION_ID").reset_index(drop=True)

<h3> Creamos la dataframes Intervenciones_Keywords </h3>


In [21]:
df_intervenciones_keywords = df[["ID_PARTICIPACION", "KEYWORDS"]].explode("KEYWORDS").dropna(subset=["KEYWORDS"])

id_participacion_to_intervencion_id = intervenciones.set_index("ID_PARTICIPACION")["INTERVENCION_ID"].to_dict()
df_intervenciones_keywords["INTERVENCION_ID"] = df_intervenciones_keywords["ID_PARTICIPACION"].map(id_participacion_to_intervencion_id)

keyword_to_id = keywords.set_index("KEYWORD")["KEYWORD_ID"].to_dict()
df_intervenciones_keywords["KEYWORD_ID"] = df_intervenciones_keywords["KEYWORDS"].map(keyword_to_id)

df_intervenciones_keywords = df_intervenciones_keywords[["INTERVENCION_ID", "KEYWORD_ID"]].dropna().drop_duplicates()

<h3> Guardamos los dataframes </h3>

In [ ]:

# DataFrame Parlamentarios
parlamentarios.to_csv("Parlamentarios.csv", index=False)

# DataFrame Partidos_Politico
partidos.to_csv("Partidos_Politico.csv", index=False)

# DataFrame Debates
debates.to_csv("Debates.csv", index=False)

# DataFrame Participaciones
participaciones.to_csv("Participaciones.csv", index=False)

# DataFrame Keywords
keywords.to_csv("Keywords.csv", index=False)

# DataFrame Intervenciones
intervenciones.to_csv("Intervenciones.csv", index=False)

# DataFrame Intervenciones_Keywords
df_intervenciones_keywords.to_csv("Intervenciones_Keywords.csv", index=False) 


'# DataFrame Parlamentarios\nparlamentarios.to_csv("Parlamentarios.csv", index=False)\n\n# DataFrame Partidos_Politico\npartidos.to_csv("Partidos_Politico.csv", index=False)\n\n# DataFrame Debates\ndebates.to_csv("Debates.csv", index=False)\n\n# DataFrame Participaciones\nparticipaciones.to_csv("Participaciones.csv", index=False)\n\n# DataFrame Keywords\nkeywords.to_csv("Keywords.csv", index=False)\n\n# DataFrame Intervenciones\nintervenciones.to_csv("Intervenciones.csv", index=False)\n\n# DataFrame Intervenciones_Keywords\ndf_intervenciones_keywords.to_csv("Intervenciones_Keywords.csv", index=False) '

## Parte 4 - [2 pts] - Informe

ver pdf: **P4_T1_Datos_Masivos.pdf**

## Parte 5 - [0.5 pts] - Resumen ejecutivo

ver pdf: **P5_T1_Datos_Masivos.pdf**